In [ ]:
# Helper function to plot data with error bars, a fit function, and its uncertainty band.

import matplotlib.pyplot as plt
import matplotlib as mpl

def plot(x_graph, y_graph, y_err, x_fit, y_fit, y_fit_lower, y_fit_upper, xlabel, ylabel = r"C [$\mu$F]"):

    # draw x_graph, y_graph with error bars
    plt.errorbar(x_graph, y_graph, y_err, fmt = 'o', label = 'Data', color = 'black')

    # draw the fit function and its uncertainty band
    plt.fill_between(x_fit, y_fit_lower, y_fit_upper, color = 'red', alpha = 0.3)

    # create a legend entry for the fit function and its uncertainty band
    line_with_band = mpl.lines.Line2D([], [], color = 'red', label = 'Fit', linestyle = '-', linewidth = 2)
    band = mpl.patches.Patch(color = 'red', alpha = 0.3, label = 'Fit uncertainty')

    # get the current legend handles and labels
    handles, labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles = handles + [(line_with_band, band)], labels = labels + ['Fit'])

    # finally, plot
    plt.plot(x_fit, y_fit, 'r-')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

    return


In [ ]:
# Solution to the homework exercise on uncertainty propagation.

import numpy as np
import scipy
from uncertainties import ufloat, unumpy

# Create numpy arrays of ufloats for y and m.
# The unumpy.uarray function creates a numpy array of ufloats from nominal values and standard deviations.
# In general, the uncertainty of each value is different, so you need to provide an array of uncertainties as the second argument.
# When all uncertainties are the same, you can provide a single value as the second argument, and it will be broadcasted to all elements.
y = unumpy.uarray([41.2, 39.6, 36.3, 33, 26.3, 16.6], 0.5) / 100
m = unumpy.uarray([150, 200, 300, 400, 600, 900], 0) / 1000

# Create the ufloat variable for the spring end position, when there is no mass attached.
y_m0 = ufloat(46.4, 0.5) / 100

# Calculate the spring prolongation, y0, propagating the uncertainties on y and y_m0.
y0 = y_m0 - y

# Create numpy arrays of ufloats for t0 and tn.
# Here, all uncertainties are the same, so we provide a single value as the second argument.
t0 = unumpy.uarray([0.28, 0.48, 0.56, 0.48, 0.24, 0.20], 0.04)
tn = unumpy.uarray([9.32, 9.84, 9.44, 14.32, 13.56, 13.12], 0.04)

# Numbers of oscillations.
n  = unumpy.uarray([20, 18, 14, 19, 15, 12], 0)

# Calculate the periods, T, and their squares, T2, propagating the uncertainties properly.
T = (tn - t0) / n
T2 = T**2

# Calculate g, propagating the uncertainties properly.
g = 4 * np.pi**2 * y0 / T2


# A simple combination of the six measurements of g is done by fitting a constant function to the g values.
def const_func(x, a):
    return a

# Perform the fit.
popt, pcov = scipy.optimize.curve_fit(const_func, 
                                      np.linspace(0, 5, 6),
                                      unumpy.nominal_values(g), 
                                      sigma = unumpy.std_devs(g), 
                                      absolute_sigma = True)
print(popt[0], '+-', np.sqrt(pcov[0,0]))

# plot the data and the fit
xfit = np.linspace(0, 5, 100)
# create numpy array with 100 identical values of popt[0]
yfit = np.full(100, popt[0])
yfit_lower = np.full(100, popt[0] - np.sqrt(pcov[0,0]))
yfit_upper = np.full(100, popt[0] + np.sqrt(pcov[0,0]))
plot(np.linspace(0, 5, 6), unumpy.nominal_values(g), unumpy.std_devs(g), 
    xfit, yfit, yfit_lower, yfit_upper, "measurement number", ylabel = r"$g$ [m/s$^2$]")


In [ ]:
# Example of a parabolic fit, and propagation of uncertainties of the (correlated) fit parameters.

import numpy as np
import scipy.optimize
import uncertainties

# Data points and the uncertainties on their y-axis coordinates.
x = np.array([3.4, 3.3, 3.25, 3.2, 3.1, 3.0])
y = np.array([-11.09, -11.271, -11.313, -11.306, -11.172, -10.817])
sigma_y = np.abs(y * 0.001)

# Define the parabolic function to fit.
def parabolic_func(x, a, b, c):
    return a * x**2 + b * x + c

# Estimate the parabolic function parameters and their covariance matrix.
popt, pcov = scipy.optimize.curve_fit(parabolic_func, x, y, sigma = sigma_y, absolute_sigma = True)
print(popt)
print(pcov)

# Create three ufloat variables, representing the fit parameters with their uncertainties and correlations.
# Note that the correlations are taken into account, and they will be considered in the uncertainty propagation.
(a, b, c) = uncertainties.correlated_values(popt, pcov)

# Plot the data and the fit function.
# ------------------------------------

# Create x values for the fit function plot.
xfit = np.linspace(2.9, 3.5, 100)

# Evaluate the fit function at the xfit points. Propagate the uncertainties properly.
yfit = parabolic_func(xfit, a, b, c) # yfit is a numpy array of ufloats

# Get nominal values and standard deviations for plotting.
yfit_nominal = unumpy.nominal_values(yfit)
yfit_stddev = unumpy.std_devs(yfit)

# Plot the data and the fit with uncertainty band.
# Note that it is not very common to plot uncertainty bands in fits like this one,
# as the uncertainty is fully determined by the covariance matrix of the fit parameters.
# In a Physics lab report, you would print the covariance matrix, but you would not usually plot the uncertainty band.
# Here, the uncertainty band is shown just to demonstrate how the propagation of correlated uncertainties works in the `uncertainties` package.
plot(x, y, sigma_y, xfit, yfit_nominal, yfit_nominal - yfit_stddev, yfit_nominal + yfit_stddev, "x", ylabel = "y")